<a href="https://colab.research.google.com/github/tradingxo/wanted_pre_onboarding/blob/main/%EC%9B%90%ED%8B%B0%EB%93%9C_%ED%94%84%EB%A6%AC%EC%98%A8%EB%B3%B4%EB%94%A9_%EC%BD%94%EC%8A%A4_%EA%B3%BC%EC%A0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#다양한 언어를 다루기 위함:Spacy
!python -m spacy download en
!python -m spacy download de

     |████████████████████████████████| 12.0 MB 7.5 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')
     |████████████████████████████████| 14.9 MB 1.4 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('de_core_news_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/de_core_news_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/de
You can now load the model via spacy.load('de')


In [ ]:
pip install torchtext

In [ ]:
import torchtext
import torch
from torchtext.data.utils import get_tokenizer
from collections import Counter
from torchtext.vocab import Vocab
from torchtext.utils import download_from_url, extract_archive
import io
import re
import itertools

# **문제 1) Tokenizer 생성하기**


**1.문장 특수문자, 소문자 변환, 공백단위 토큰화**

**2.전처리 함수 이용 토큰화, 정수인덱싱 위한 어휘사전 생성**

**3.어휘사전에 없는 단어는 'OOV'의 index로 변환**

In [ ]:
class Tokenizer():
  def __init__(self):
    self.word_dict = {'oov': 0}
    self.fit_checker = False
  
  def preprocessing(self, sequences):
      result = []
      '''
      문제 1-1.
      '''
      for s in sequences:
          # 특수문자 제거
          s =re.sub(r"[^a-zA-Z0-9","",s)
          # 소문자 변환 후 토큰화
          s =s.lower().split()
          result.append(s)
          return result
  
  def fit(self, sequences):
    self.fit_checker = False
    '''
    문제 1-2.
    '''
    tokens = self.preprocessing(sequences)
    #전처리 함수 토큰화
    tokens = list(itertools.chain(*tokens)) #중첩 리스트 해제
    tokens = list(dict.fromkeys(tokens)) #중복 제거

    #정수 인덱싱(특별 규칙 없이 순서대로 정수 부여)
    num =1
    for t in tokens:
        self.word_dict[t] = num
        num += 1
        self.fit_checker = True
  
  def transform(self, sequences):
    result = []
    tokens = self.preprocessing(sequences)
    if self.fit_checker:
      '''
      문제 1-3.
      '''
    for token in tokens :
            num_list = []
            for t in token :
                # 단어(t)가 word_dict에 있을 경우, value를 추가 
                if t in self.word_dict.keys():
                    num_list.append(self.word_dict[t])
                # 없을 경우, oov의 value를 추가
                else:
                    num_list.append(self.word_dict['oov'])
            result.append(num_list)
            return result
    else:
        raise Exception("Tokenizer instance is not fitted yet.")
      
  def fit_transform(self, sequences):
    self.fit(sequences)
    result = self.transform(sequences)
    return result

# **문제 2) TfidfVectorizer 생성하기**


**1.IDF행렬은 list형태, IDF식 참조, 문제1의 Tokenizer사용**

**2.입력 문장을 이용해 TF행렬, ( fit())에서 만든 IDF 행렬과 아래 식을 이용해 TF-IDF 행렬 만들기**

In [ ]:
from math import log 
import numpy as np

class TfidfVectorizer:
  def __init__(self, tokenizer):
    self.tokenizer = tokenizer
    self.fit_checker = False
  
  def fit(self, sequences):
    tokenized = self.tokenizer.fit_transform(sequences)
    '''
    문제 2-1.
    '''
    n = len(tokenized) # 총 문서의 수
    self.idf=[]
    self.tokens = list(itertools.chain(*tokenized)) # 중첩된 리스트 해제 
    self.tokens = list(dict.fromkeys(self.tokens)) # 중복 제거
    for t in self.tokens:
      df = 0
      # 만약 문장(d)에 토큰(t)가 있다면 1(True)추가
      for d in tokenized:
        df += t in d
      self.idf.append(log(n/(df+1)))
    self.fit_checker = True
    
  def transform(self, sequences):
    if self.fit_checker:
      tokenized = self.tokenizer.transform(sequences)
      '''
      문제 2-2.
      '''
      # tf 구하기
      tf = []
      for d in tokenized:
        frequency = []
        for t in self.tokens:
          frequency.append(d.count(t))
        tf.append(frequency)

      # array끼리 곱
      idf  = np.array(self.idf)
      tf = np.array(tf)
      self.tfidf_matrix = tf*idf

      # nested list로 변환
      self.tfidf_matrix = self.tfidf_matrix.tolist()

      return self.tfidf_matrix
    else:
      raise Exception("TfidfVectorizer instance is not fitted yet.")

  def fit_transform(self, sequences):
    self.fit(sequences)
    return self.transform(sequences)